# In this notebook
I run the code in this repository outside of a script for debugging purposes.

Note that this directory has moved, so any relative references may be broken.

In [1]:
import os
from itertools import chain

import torch
from torch.utils.data import DataLoader

from src import datasets, utils, metrics
from models import encoders, decoders

In [3]:
train_data, test_data = datasets.load('solar')
train_loader = DataLoader(train_data, batch_size=8, shuffle=True)
test_loader = DataLoader(test_data, batch_size=8, shuffle=False)

encoder = encoders.load('swav')
decoder = decoders.load('unet', encoder)

Using cache found in /home/zdc6/.cache/torch/hub/facebookresearch_swav_main


In [4]:
if torch.cuda.is_available():
	device = 'cuda'
else:
	device = 'cpu'

print(device)

encoder = encoder.to(device)
decoder = decoder.to(device)

cuda


In [5]:
optimizer = torch.optim.Adam(decoder.parameters(), lr=0.001, weight_decay=0.0)
criterion = metrics.load('xent', device)

In [6]:

encoder.eval()

decoder.train()
criterion = criterion.cuda()
avg_loss = utils.AverageMeter()
num_batches = len(train_loader)
for batch_idx, (inp, target) in enumerate(train_loader):
	if batch_idx % 10 == 0:
		print(f"Beginning batch {batch_idx} of {num_batches}")
	# Move to the GPU
	inp = inp.cuda()
	target = target.cuda()

	with torch.no_grad():
		output = encoder(inp)

	output = decoder(output)

	loss = criterion(output, target)

	# Calculate the gradients
	optimizer.zero_grad()
	loss.backward()
	avg_loss.update(loss.item(), inp.size(0))
	# Step forward
	optimizer.step()

print(avg_loss)

Beginning batch 0 of 42189


/usr/lib/python3/dist-packages/torch/nn/functional.py:3631: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn(


Beginning batch 10 of 42189


KeyboardInterrupt: 